In [ ]:
#RUN process_result_csvs.ipynb first to process the csvs before plotting the mean similarities here
import pandas as pd
import matplotlib.pyplot as plt
import re

In [ ]:
mammo="" #Path to processed mammo csv from process_result_csvs.ipynb
df_mammo = pd.read_csv(mammo)
unique_layers_mammo = df_mammo['layer'].unique()

In [ ]:
mean_sims=[]
for layer in unique_layers_mammo:
    # Filter the DataFrame to get rows corresponding to the current layer
    layer_data = df_mammo[df_mammo['layer'] == layer]
    #use column Mean_similarity to get mean similarity
    mean_layer_sim= layer_data['Mean_similarity'].mean()
    mean_sims.append(mean_layer_sim)

In [ ]:
print(mean_sims)
print(len(mean_sims))

In [ ]:
gen="" #Path to processed general csv from process_result_csvs.ipynb
df_gen = pd.read_csv(gen)
unique_layers_gen= df_gen['layer'].unique()

In [ ]:
mean_sims_gen = []
for layer in unique_layers_gen:
    # Filter the DataFrame to get rows corresponding to the current layer
    layer_data = df_gen[df_gen['layer'] == layer]
    #use coloumn Mean_similarity to get mean similarity
    mean_layer_sim= layer_data['Mean_similarity'].mean()
    mean_sims_gen.append(mean_layer_sim)

In [ ]:
print(mean_sims_gen)

In [ ]:
#sort unique layers
unique_layers_mammo = sorted(unique_layers_mammo, key=lambda x: int(re.search(r'\d+', x).group()))
unique_layers_gen = sorted(unique_layers_gen, key=lambda x: int(re.search(r'\d+', x).group()))
# Use integer positions for the x-axis
x_positions = range(len(unique_layers_mammo))  # Positions for unique_layers
x_positions2 = range(len(unique_layers_gen))  # Positions for unique_layers2
plt.figure(figsize=[10,6])
# Plot the data
plt.plot(x_positions, mean_sims, marker='o', label='M-Mammo-CLIP Dissect',color='#87CEEB')
plt.plot(x_positions2, mean_sims_gen, marker='s', label='G-Mammo-CLIP Dissect',color='#8B008B')
# Add labels, title, and legend
plt.xlabel('Layer', fontsize=14)
plt.ylabel('Mean Similarity', fontsize=14)
plt.ylim(0, 2.7)
#plt.title('Comparing Locally Thresholded Mean Similarity for Each Layer')
plt.legend(loc='best', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.savefig("YourDir")
plt.show()

In [ ]:
#FIGURE 4B
# Paths to the CSV files
csv_mammo_path = "/Users/ssa195/Library/CloudStorage/OneDrive-UiTOffice365/Documents/lumi_mammo_dissect/UPDATED_CSVS/new_mammoclip_mammo_csvs/Global_Thresh_MammoCLIP_Cat_Vindr_mammo_pretrained_breast_clip_descriptions_sorted.csv"  # Replace with actual path
csv_gen_path = "/Users/ssa195/Library/CloudStorage/OneDrive-UiTOffice365/Documents/lumi_mammo_dissect/UPDATED_CSVS/new_mammoclip_mammo_csvs/Global_Thresh_MammoCLIP_Cat_Vindr_gen_pretrained_breast_clip_descriptions_sorted.csv"    # Replace with actual path
# Load the CSV files
df1 = pd.read_csv(csv_gen_path,dtype=str )  # General pretrained model concepts
df2 = pd.read_csv(csv_mammo_path, dtype=str)  # Mammo pretrained model concepts
# Filter the DataFrames to only include rows where Local_Threshold_met is True
df1 = df1[df1['Global_thresh_met'] == 'True']
df2 = df2[df2['Global_thresh_met'] == 'True']
# Define the two categories
mammography_concepts = "Mammography concepts"
non_mammography_concepts = "Non-mammography concepts"
# Create a new column to classify Broad_category into the two categories
for df in [df1, df2]:
    df['Category'] = df['Broad_categories'].apply(
        lambda x: mammography_concepts if x != "Environmental_and_Natural" else non_mammography_concepts
    )
# Define the specific layers of interest
layers_of_interest = ["image_encoder._blocks[2]", "image_encoder._blocks[19]", "image_encoder._blocks[38]"]
# Define custom labels for the layers
layer_labels = {
    "image_encoder._blocks[2]": "Block 1",
    "image_encoder._blocks[19]": "Block 4",
    "image_encoder._blocks[38]": "Block 7"
}
# Filter the DataFrames to only include the layers of interest
df1 = df1[df1['layer'].isin(layers_of_interest)]
df2 = df2[df2['layer'].isin(layers_of_interest)]
# Create dictionaries to store the counts for each layer
counts_dict1 = {layer: {mammography_concepts: 0, non_mammography_concepts: 0} for layer in layers_of_interest}
counts_dict2 = {layer: {mammography_concepts: 0, non_mammography_concepts: 0} for layer in layers_of_interest}
# Count occurrences for df1 (general pretrained model concepts)
for index, row in df1.iterrows():
    layer = row['layer']
    category = row['Category']
    counts_dict1[layer][category] += 1
# Count occurrences for df2 (mammo pretrained model concepts)
for index, row in df2.iterrows():
    layer = row['layer']
    category = row['Category']
    counts_dict2[layer][category] += 1
# Prepare data for the bar plots
layers = layers_of_interest
mammography_counts1 = [counts_dict1[layer][mammography_concepts] for layer in layers]
non_mammography_counts1 = [counts_dict1[layer][non_mammography_concepts] for layer in layers]
mammography_counts2 = [counts_dict2[layer][mammography_concepts] for layer in layers]
non_mammography_counts2 = [counts_dict2[layer][non_mammography_concepts] for layer in layers]
# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
# Bar width
width = 0.35
x = np.arange(len(layers))  # The label locations
# Plot Mammography-related concepts
bars1 = axes[0].bar(x - width/2, mammography_counts1, width, label='G-Mammo-CLIP Dissect', color='#8B008B')
bars2 = axes[0].bar(x + width/2, mammography_counts2, width, label='M-Mammo-CLIP Dissect', color='#87CEEB')
axes[0].set_title('Mammography-related Concepts', fontsize=18)  # Increased font size for title
axes[0].set_ylabel('Number of Mammography Concepts', fontsize=14)  # Increased font size for y-axis label
axes[0].set_xticks(x)
axes[0].set_xticklabels([layer_labels[layer] for layer in layers], rotation=0, fontsize=16)  # Increased font size for tick labels
axes[0].legend(loc='upper left', fontsize=14)  # Increased font size for legend
axes[0].grid(axis='y', linestyle='--', alpha=0.7)  # Add faint grid lines
# Set y-axis limits for the first subplot
axes[0].set_ylim(0, 215)
# Add numbers on top of bars for Mammography-related concepts
for bar in bars1:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2, height, f'{int(height)}', ha='center', va='bottom', fontsize=16)  # Increased font size
for bar in bars2:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2, height, f'{int(height)}', ha='center', va='bottom', fontsize=16)  # Increased font size
# Plot Non-mammography-related concepts
bars3 = axes[1].bar(x - width/2, non_mammography_counts1, width, label='G-Mammo-CLIP Dissect', color='#8B008B')
bars4 = axes[1].bar(x + width/2, non_mammography_counts2, width, label='M-Mammo-CLIP Dissect', color='#87CEEB')
axes[1].set_title('Non-Mammography-related Concepts', fontsize=18)  # Increased font size for title
axes[1].set_ylabel('Number of Non-Mammography Concepts', fontsize=14)  # Increased font size for y-axis label
axes[1].set_xticks(x)
axes[1].set_xticklabels([layer_labels[layer] for layer in layers], rotation=0, fontsize=16)  # Increased font size for tick labels
axes[1].legend(loc='upper left', fontsize=14)  # Increased font size for legend
axes[1].grid(axis='y', linestyle='--', alpha=0.7)  # Add faint grid lines
# Set y-axis limits for the second subplot
axes[1].set_ylim(0, 215)
# Add numbers on top of bars for Non-mammography-related concepts
for bar in bars3:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2, height, f'{int(height)}', ha='center', va='bottom', fontsize=16)  # Increased font size
for bar in bars4:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2, height, f'{int(height)}', ha='center', va='bottom', fontsize=16)  # Increased font size
# Adjust layout
plt.tight_layout()
# Save the plot
output_plot_path = ""  # Replace with actual path
plt.savefig(output_plot_path, dpi=300, bbox_inches='tight')
print(f"Plot saved to: {output_plot_path}")
# Show the plot
plt.show()

In [ ]:
#FIGURE 4C
# Paths to the CSV files
csv_mammo_path = "/Users/ssa195/Library/CloudStorage/OneDrive-UiTOffice365/Documents/lumi_mammo_dissect/UPDATED_CSVS/new_mammoclip_mammo_csvs/Global_Thresh_MammoCLIP_Cat_Vindr_mammo_pretrained_breast_clip_descriptions_sorted.csv"  # Replace with actual path
csv_gen__path = "/Users/ssa195/Library/CloudStorage/OneDrive-UiTOffice365/Documents/lumi_mammo_dissect/UPDATED_CSVS/new_mammoclip_mammo_csvs/Global_Thresh_MammoCLIP_Cat_Vindr_gen_pretrained_breast_clip_descriptions_sorted.csv"    # Replace with actual path
# Load the CSV files
df1 = pd.read_csv(csv_gen_path, dtype=str)  # General pretrained model concepts
df2 = pd.read_csv(csv_mammo_path, dtype=str)  # Mammo pretrained model concepts
# Filter the DataFrames to only include rows where Local_Threshold_met is True
df1 = df1[df1['Global_thresh_met'] == 'True']
df2 = df2[df2['Global_thresh_met'] == 'True']
# Define the specific layers of interest
layers_of_interest = ["image_encoder._blocks[2]", "image_encoder._blocks[19]", "image_encoder._blocks[38]"]
# Define custom labels for the layers
layer_labels = {
    "image_encoder._blocks[2]": "Block 1",
    "image_encoder._blocks[19]": "Block 4",
    "image_encoder._blocks[38]": "Block 7"
}
# Filter the DataFrames to only include the layers of interest
df1 = df1[df1['layer'].isin(layers_of_interest)]
df2 = df2[df2['layer'].isin(layers_of_interest)]
# Define the color mapping for Broad_categories
color_mapping = {
    "Breast_anatomy_or_structure": "brown",
    "Breast_locations": "orange",
    "Findings and characterizations": "violet",
    "Interpretations": "blue",
    "Action or follow up": "red",
    "Environmental_and_Natural": "green",
    "Miscellaneous": "gray"
}
# Initialize dictionaries to store top 3 Broad_categories for each layer
top_categories_dict1 = {}
top_categories_dict2 = {}
# Function to calculate top 3 Broad_categories for a DataFrame
def calculate_top_categories(df, layers):
    top_categories = {}
    for layer in layers:
        layer_df = df[df['layer'] == layer]
        category_counts = layer_df['Broad_categories'].value_counts()
        top_categories[layer] = category_counts.head(3)  # Get top 3 categories
    return top_categories
# Calculate top 3 Broad_categories for each layer in both DataFrames
top_categories_dict1 = calculate_top_categories(df1, layers_of_interest)
top_categories_dict2 = calculate_top_categories(df2, layers_of_interest)
# Prepare data for the stacked bar plots
x = np.arange(len(layers_of_interest))  # The label locations
width = 0.6  # The width of the bars
# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
# Set y-axis limits for both subplots
axes[0].set_ylim(0, 215)  # First subplot
axes[1].set_ylim(0, 215)  # Second subplot
# Add faint grid lines to both subplots
for ax in axes:
    ax.grid(axis='y', linestyle='--', alpha=0.7)  # Add grid lines to the y-axis with dashed lines and reduced opacity
# Plot for general pretrained (csv1)
handles = []  # To store legend handles
labels = []   # To store legend labels
for i, layer in enumerate(layers_of_interest):
    general_values = top_categories_dict1[layer].values if layer in top_categories_dict1 else [0, 0, 0]
    general_labels = top_categories_dict1[layer].index if layer in top_categories_dict1 else ["None", "None", "None"]
    bottom = 0
    for value, label in zip(general_values, general_labels):
        color = color_mapping.get(label.strip(), "gray")  # Use color mapping, default to gray
        bar = axes[0].bar(x[i], value, width, label=label, bottom=bottom, color=color)
        bottom += value
        if label not in labels:  # Avoid duplicate legend entries
            handles.append(bar)
            labels.append(label)
axes[0].set_title('G-Mammo-CLIP Dissect', fontsize=16)
#axes[0].set_xlabel('Layer',fontsize=12)
axes[0].set_ylabel('Number of concepts', fontsize=16)
axes[0].set_xticks(x)
axes[0].set_xticklabels([layer_labels[layer] for layer in layers_of_interest], rotation=0, fontsize=16)
# Plot for mammo pretrained (csv2)
for i, layer in enumerate(layers_of_interest):
    mammo_values = top_categories_dict2[layer].values if layer in top_categories_dict2 else [0, 0, 0]
    mammo_labels = top_categories_dict2[layer].index if layer in top_categories_dict2 else ["None", "None", "None"]
    bottom = 0
    for value, label in zip(mammo_values, mammo_labels):
        color = color_mapping.get(label.strip(), "gray")  # Use color mapping, default to gray
        bar = axes[1].bar(x[i], value, width, label=label, bottom=bottom, color=color)
        bottom += value
        if label not in labels:  # Avoid duplicate legend entries
            handles.append(bar)
            labels.append(label)
axes[1].set_title('M-Mammo-CLIP Dissect', fontsize=16)
#axes[1].set_xlabel('Layer', fontsize=12)
axes[1].set_xticks(x)
axes[1].set_xticklabels([layer_labels[layer] for layer in layers_of_interest], rotation=0, fontsize=16)
# Remove underscores from legend labels
clean_labels = [label.replace("_", " ") for label in labels]
# Add a single legend for both subplots
fig.legend(handles, clean_labels, loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3, fontsize=16)
# Adjust layout
plt.tight_layout()
# Save the plot
output_plot_path = ""  # Replace with actual path
plt.savefig(output_plot_path, dpi=300, bbox_inches='tight')
print(f"Plot saved to: {output_plot_path}")
# Show the plot
plt.show()